<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
ELEC PRICE PREDICTION CAPSTONE:</font>  

<font size='4'>Day Ahead Market RNN Model for Continuous Target</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Manu Kalia Project Submission<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; DSI-7-SF<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15-May-2019<br>
</font>

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Function-Definitions" data-toc-modified-id="Function-Definitions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function Definitions</a></span></li><li><span><a href="#Load-Pickles:--Train/Test-Dataframes-&amp;--Scaled-Arrays" data-toc-modified-id="Load-Pickles:--Train/Test-Dataframes-&amp;--Scaled-Arrays-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Pickles:  Train/Test Dataframes &amp;  Scaled Arrays</a></span></li><li><span><a href="#Create-(scaled)-Train-test-X-and-y-Pairs-for-Day-Ahead-RNN-Model" data-toc-modified-id="Create-(scaled)-Train-test-X-and-y-Pairs-for-Day-Ahead-RNN-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create (scaled) Train-test X and y Pairs for Day Ahead RNN Model</a></span></li><li><span><a href="#Time-Series-Generator" data-toc-modified-id="Time-Series-Generator-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Time Series Generator</a></span></li><li><span><a href="#RNN-Model" data-toc-modified-id="RNN-Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>RNN Model</a></span></li></ul></div>

---

## Imports

In [ ]:
import pandas as pd
import numpy as np

import pickle
import wget, os
import time
import glob
import pytz

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
# 
register_matplotlib_converters()
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU
from keras.optimizers import Adam
from keras.preprocessing.sequence import TimeseriesGenerator

---

## Function Definitions

---

## Load Pickles:  Train/Test Dataframes &  Scaled Arrays

In [ ]:
with open('../data/processed/train.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('../data/processed/test.pkl', 'rb') as f:
    test = pickle.load(f)

---

## Create (scaled) Train-test X and y Pairs for Day Ahead RNN Model

In [ ]:
X_train = train[train.column != 'dam_price_per_mwh']
y_train = train['dam_price_per_mwh']

X_test = test[test.column != 'dam_price_per_mwh']
y_test = test['dam_price_per_mwh']

In [ ]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

---

## Time Series Generator

In [ ]:
train_sequences  = TimeseriesGenerator(X_train_sc, y_train, length=3, batch_size=64)
batch_x, batch_y = train_sequences[0]

In [ ]:
print(batch_x.shape)
print(batch_y.shape)

In [ ]:
pd.DataFrame(X_train_sc, columns=X_train.columns).head(3)

In [ ]:
test_sequences = TimeseriesGenerator(X_test_sc, y_test, length=3, batch_size=64)

---

## RNN Model

In [ ]:
model = Sequential()
model.add(GRU(8, input_shape=(3, 8), return_sequences=True))
model.add(GRU(8))

model.add(Dense(4, activation='relu'))

model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(optimizer=Adam(lr=.0005),
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

history = model.fit_generator(train_sequences,
                              validation_data=test_sequences,
                              epochs=20,
                              verbose=1)

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']

plt.figure(figsize=(9, 5))
plt.plot(train_loss, label='Training loss', color='darkblue')
plt.plot(test_loss, label='Testing loss', color='darkorange')
plt.legend();

In [ ]:
train_mse = history.history['mean_squared_error']
test_mse = history.history['val_mean_squared_error']

plt.figure(figsize=(9, 5))
plt.plot(train_mse, label='Training mse', color='darkblue')
plt.plot(test_mse, label='Testing mse', color='darkorange')
plt.legend();